In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import lightgbm as lgbm
#from xgboost import plot_importance
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%%time
is_valid = False

path="./mnt/ssd/kaggle-talkingdata2/competition_files/"

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
#test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

# Read the last lines because they are more impacting in training than the starting lines
train = pd.read_csv(path+"train.csv", skiprows=range(1,131886954), nrows=53016937, usecols=train_columns, dtype=dtypes)
#valid = pd.read_csv(path+"test.csv", usecols=test_columns, dtype=dtypes)


CPU times: user 1min 23s, sys: 2.99 s, total: 1min 26s
Wall time: 1min 26s


In [3]:

gc.collect()

7

In [4]:
train.tail()

,ip,app,device,os,channel,click_time,is_attributed
53016932,121312,12,1,10,340,2017-11-09 16:00:00,0
53016933,46894,3,1,19,211,2017-11-09 16:00:00,0
53016934,320126,1,1,13,274,2017-11-09 16:00:00,0
53016935,189286,12,1,37,259,2017-11-09 16:00:00,0
53016936,106485,11,1,19,137,2017-11-09 16:00:00,0


In [ ]:
#Y=train.is_attributed
#train.drop('is_attributed',axis=1,inplace=True)

In [5]:
def timeFeatures(df,colname):
    # Make some new features with click_time column
    df['datetime'] = pd.to_datetime(df[colname])
    df['dow']      = df['datetime'].dt.dayofweek
    df["hour"]      = df['datetime'].dt.hour
    #df["dteom"]    = df["datetime"].dt.daysinmonth - df["datetime"].dt.day
    df.drop(['datetime'], axis=1, inplace=True)
    return df


In [6]:
%%time
#valid=timeFeatures(valid)
train=timeFeatures(train,'click_time')
gc.collect()

CPU times: user 18.3 s, sys: 1.74 s, total: 20 s
Wall time: 19.9 s


In [ ]:
'''
nrow_train = train.shape[0]
merge = pd.concat([train, test])
gc.collect()
'''

In [7]:
valid=pd.DataFrame()
valid=train[train.hour>=15]
train=train[train.hour<15]
gc.collect()
valid.shape,train.shape

((3026558, 9), (49990379, 9))

In [8]:
train.groupby(['ip','device','os'])['app'].agg(['mean','count']).reset_index().head()

,ip,device,os,mean,count
0,1,1,2,10.6,5
1,1,1,4,7.5,2
2,1,1,9,5.8,5
3,1,1,27,5.0,1
4,1,1,30,7.5,2


In [9]:
gc.collect()

58

In [10]:
train.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour
0,201143,11,1,13,487,2017-11-09 00:00:00,0,3,0
1,34684,2,1,13,469,2017-11-09 00:00:00,0,3,0
2,207368,26,1,19,477,2017-11-09 00:00:00,0,3,0
3,110176,18,1,8,121,2017-11-09 00:00:00,0,3,0
4,109644,12,1,19,265,2017-11-09 00:00:00,0,3,0


In [13]:
%%time
###Experiment
def get_feature_count(train_df,valid_df):
    

    nrow_train = train_df.shape[0]
    df = pd.concat([train_df, valid_df])
    gc.collect()

    del train_df,valid_df
    
    ip_count = df.groupby(['ip','device','os','app'])['channel'].count().reset_index()
    ip_count.columns = ['ip','device','os','app', 'clicks_by_ip_os_device_app']
    df = pd.merge(df, ip_count, on=['ip','device','os','app'], how='left', sort=False)
    #valid_df = pd.merge(valid_df, ip_count, on=['ip','device','os','app'], how='left', sort=False)
    del ip_count
    gc.collect()
    print('done')
    
    '''
    ip_count = df.groupby(['device','os','app'])['channel'].count().reset_index()
    ip_count.columns = ['device','os','app', 'clicks_by_os_device_app']
    df = pd.merge(df, ip_count, on=['device','os','app'], how='left', sort=False)
    #valid_df = pd.merge(valid_df, ip_count, on=['device','os','app'], how='left', sort=False)
    #df['clicks_by_os_device_app'] = df['clicks_by_os_device_app'].astype('uint16')
    #valid_df['clicks_by_os_device_app'] = valid_df['clicks_by_os_device_app'].astype('uint16')
    del ip_count
    gc.collect()
    print('done')
    '''
    
    ip_count = df.groupby(['ip','device','os'])['app'].agg(['count']).reset_index()
    ip_count.columns = ['ip','device','os','count_app_by_iposdevice']
    df = pd.merge(df, ip_count, on=['ip','device','os'], how='left', sort=False)
    #valid_df = pd.merge(valid_df, ip_count, on=['ip','device','os'], how='left', sort=False)
    del ip_count
    gc.collect()

    # Count the number of clicks by ip
    ip_count = df.groupby(['ip'])['channel'].count().reset_index()
    ip_count.columns = ['ip', 'clicks_by_ip']
    df = pd.merge(df, ip_count, on='ip', how='left', sort=False)
    #valid_df = pd.merge(valid_df, ip_count, on='ip', how='left', sort=False)
    #df['clicks_by_ip'] = df['clicks_by_ip'].astype('uint16')
    #valid_df['clicks_by_ip'] = valid_df['clicks_by_ip'].astype('uint16')
    del ip_count
    gc.collect()
    print('done')

    ip_count = df.groupby(['ip','app'])['channel'].count().reset_index()
    ip_count.columns = ['ip','app', 'clicks_by_ip_app']
    df = pd.merge(df, ip_count, on=['ip','app'], how='left', sort=False)
    #df['clicks_by_ip_app'] = df['clicks_by_ip_app'].astype('uint16')
    #valid_df = pd.merge(valid_df, ip_count, on=['ip','app'], how='left', sort=False)
    #valid_df['clicks_by_ip_app'] = valid_df['clicks_by_ip_app'].astype('uint16')
    del ip_count
    gc.collect()
    print('done')

    ip_count = df.groupby(['ip','os'])['channel'].count().reset_index()
    ip_count.columns = ['ip','os', 'clicks_by_ip_os']
    df = pd.merge(df, ip_count, on=['ip','os'], how='left', sort=False)
    #df['clicks_by_ip_os'] = df['clicks_by_ip_os'].astype('uint16')
    #valid_df = pd.merge(valid_df, ip_count, on=['ip','os'], how='left', sort=False)
    #valid_df['clicks_by_ip_os'] = valid_df['clicks_by_ip_os'].astype('uint16')
    
    del ip_count
    

    train_df = df[:nrow_train]
    valid_df = df[nrow_train:]

    del df
    print('done')
    #merge.drop('ip', axis=1, inplace=True)
    gc.collect()
    return train_df,valid_df


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.11 µs


In [14]:
%%time
gc.collect()
train,valid=get_feature_count(train,valid)
print(valid.shape,train.shape)

done
done
done
done
done
(3026558, 15) (49990379, 15)
CPU times: user 2min 28s, sys: 35.3 s, total: 3min 3s
Wall time: 3min 3s


In [16]:
train.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,clicks_by_os_device_app,count_app_by_iposdevice,clicks_by_ip,clicks_by_ip_app,clicks_by_ip_os
0,201143,11,1,13,487,2017-11-09 00:00:00,0,3,0,13,240248,198,973,56,198
1,34684,2,1,13,469,2017-11-09 00:00:00,0,3,0,5,1213714,130,836,58,130
2,207368,26,1,19,477,2017-11-09 00:00:00,0,3,0,13,264710,319,1079,27,319
3,110176,18,1,8,121,2017-11-09 00:00:00,0,3,0,9,152110,302,1864,163,302
4,109644,12,1,19,265,2017-11-09 00:00:00,0,3,0,239,1452671,1992,8677,1054,2199


In [17]:

def get_feature_user(df,valid_df):
        
    ip_count = df.groupby(['ip','device','os'])['is_attributed'].agg(['mean']).reset_index()
    ip_count.columns = ['ip','device','os', 'mean_attr_by_iposdevice']
    df = pd.merge(df, ip_count, on=['ip','device','os'], how='left', sort=False)
    valid_df = pd.merge(valid_df, ip_count, on=['ip','device','os'], how='left', sort=False)
    del ip_count
    gc.collect()
    
    agg_cols = ['device', 'app', 'channel','ip','os'];
    for c in agg_cols:
        gp = df.groupby(c)['is_attributed']
        mean = gp.mean()

        df[c + '_attr_avg'] = df[c].map(mean)
        
        valid_df[c + '_attr_avg'] = valid_df[c].map(mean)
        
    
    del mean,gp,agg_cols
    return df,valid_df
    

In [18]:
%%time
train,valid=get_feature_user(train,valid)
print(valid.shape,train.shape)

(3026558, 21) (49990379, 21)
CPU times: user 1min 14s, sys: 14.8 s, total: 1min 29s
Wall time: 1min 29s


In [19]:
print(valid.shape,train.shape)

(3026558, 21) (49990379, 21)


In [20]:
train.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,...,count_app_by_iposdevice,clicks_by_ip,clicks_by_ip_app,clicks_by_ip_os,mean_attr_by_iposdevice,device_attr_avg,app_attr_avg,channel_attr_avg,ip_attr_avg,os_attr_avg
0,201143,11,1,13,487,2017-11-09 00:00:00,0,3,0,13,...,198,973,56,198,0.000000,0.001747,0.001551,0.001501,0.001064,0.001490
1,34684,2,1,13,469,2017-11-09 00:00:00,0,3,0,5,...,130,836,58,130,0.000000,0.001747,0.000220,0.000417,0.001332,0.001490
2,207368,26,1,19,477,2017-11-09 00:00:00,0,3,0,13,...,319,1079,27,319,0.000000,0.001747,0.000388,0.000202,0.001871,0.001755
3,110176,18,1,8,121,2017-11-09 00:00:00,0,3,0,9,...,302,1864,163,302,0.000000,0.001747,0.000496,0.000390,0.001670,0.001042
4,109644,12,1,19,265,2017-11-09 00:00:00,0,3,0,239,...,1992,8677,1054,2199,0.000537,0.001747,0.000103,0.000150,0.000501,0.001755


In [ ]:
'''
%%time
ip_count = merge.groupby(['ip','dow','hour'])['channel'].count().reset_index()
ip_count.columns = ['ip','dow','hour','clicks_by_ip_dow_hour']
merge = pd.merge(merge, ip_count, on=['ip','dow','hour'], how='left', sort=False)
merge['clicks_by_ip_dow_hour'] = merge['clicks_by_ip_dow_hour'].astype('uint16')
del ip_count
gc.collect()

'''

In [ ]:
'''
%%time

# Count the number of os by ip
ip_count = merge.groupby(['channel'])['os'].count().reset_index()
ip_count.columns = ['channel', 'clicks_by_channel']
merge = pd.merge(merge, ip_count, on='channel', how='left', sort=False)
merge['clicks_by_channel'] = merge['clicks_by_channel'].astype('uint16')
#merge.drop('channel', axis=1, inplace=True)
del ip_count
print('channel done')

ip_count = merge.groupby(['os'])['channel'].count().reset_index()
ip_count.columns = ['os', 'clicks_by_os']
merge = pd.merge(merge, ip_count, on='os', how='left', sort=False)
merge['clicks_by_os'] = merge['clicks_by_os'].astype('uint16')
#merge.drop('os', axis=1, inplace=True)
del ip_count
print('os done')
gc.collect()

ip_count = merge.groupby(['app'])['channel'].count().reset_index()
ip_count.columns = ['app', 'clicks_by_app']
merge = pd.merge(merge, ip_count, on='app', how='left', sort=False)
merge['clicks_by_app'] = merge['clicks_by_app'].astype('uint16')
#merge.drop('app', axis=1, inplace=True)
del ip_count

print('app done')
#del ip_count,merge
#gc.collect()
'''

In [ ]:
'''

train = merge[:nrow_train]
test = merge[nrow_train:]

del merge

gc.collect()
'''

In [ ]:
#test.head()

In [26]:
train.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,...,count_app_by_iposdevice,clicks_by_ip,clicks_by_ip_app,clicks_by_ip_os,mean_attr_by_iposdevice,device_attr_avg,app_attr_avg,channel_attr_avg,ip_attr_avg,os_attr_avg
0,201143,11,1,13,487,2017-11-09 00:00:00,0,3,0,13,...,198,973,56,198,0.000000,0.001747,0.001551,0.001501,0.001064,0.001490
1,34684,2,1,13,469,2017-11-09 00:00:00,0,3,0,5,...,130,836,58,130,0.000000,0.001747,0.000220,0.000417,0.001332,0.001490
2,207368,26,1,19,477,2017-11-09 00:00:00,0,3,0,13,...,319,1079,27,319,0.000000,0.001747,0.000388,0.000202,0.001871,0.001755
3,110176,18,1,8,121,2017-11-09 00:00:00,0,3,0,9,...,302,1864,163,302,0.000000,0.001747,0.000496,0.000390,0.001670,0.001042
4,109644,12,1,19,265,2017-11-09 00:00:00,0,3,0,239,...,1992,8677,1054,2199,0.000537,0.001747,0.000103,0.000150,0.000501,0.001755


In [31]:
train["click_time"] = pd.to_datetime(train["click_time"], format="%Y-%m-%d %H:%M:%S")
valid["click_time"] = pd.to_datetime(valid["click_time"], format="%Y-%m-%d %H:%M:%S")
train.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,...,count_app_by_iposdevice,clicks_by_ip,clicks_by_ip_app,clicks_by_ip_os,mean_attr_by_iposdevice,device_attr_avg,app_attr_avg,channel_attr_avg,ip_attr_avg,os_attr_avg
0,201143,11,1,13,487,2017-11-09,0,3,0,13,...,198,973,56,198,0.000000,0.001747,0.001551,0.001501,0.001064,0.001490
1,34684,2,1,13,469,2017-11-09,0,3,0,5,...,130,836,58,130,0.000000,0.001747,0.000220,0.000417,0.001332,0.001490
2,207368,26,1,19,477,2017-11-09,0,3,0,13,...,319,1079,27,319,0.000000,0.001747,0.000388,0.000202,0.001871,0.001755
3,110176,18,1,8,121,2017-11-09,0,3,0,9,...,302,1864,163,302,0.000000,0.001747,0.000496,0.000390,0.001670,0.001042
4,109644,12,1,19,265,2017-11-09,0,3,0,239,...,1992,8677,1054,2199,0.000537,0.001747,0.000103,0.000150,0.000501,0.001755


In [ ]:
%%time
train["ordinal_date"] = train["click_time"].apply(lambda x: time.mktime(x.timetuple()))
valid["ordinal_date"] = valid["click_time"].apply(lambda x: time.mktime(x.timetuple()))

In [35]:
train["train_set"] = 1
valid["train_set"] = 0
gc.collect()

234

In [36]:
%%time
train = train.sort_values(by="ordinal_date").reset_index(drop=True)
valid = valid.sort_values(by="ordinal_date").reset_index(drop=True)

CPU times: user 10.5 s, sys: 6.5 s, total: 17 s
Wall time: 17 s


In [37]:
train.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,...,clicks_by_ip_app,clicks_by_ip_os,mean_attr_by_iposdevice,device_attr_avg,app_attr_avg,channel_attr_avg,ip_attr_avg,os_attr_avg,ordinal_date,train_set
0,201143,11,1,13,487,2017-11-09,0,3,0,13,...,56,198,0.000000,0.001747,0.001551,0.001501,0.001064,0.001490,1.510186e+09,1
1,106107,14,1,19,439,2017-11-09,0,3,0,34,...,151,853,0.001289,0.001747,0.000247,0.000447,0.002257,0.001755,1.510186e+09,1
2,73516,12,1,19,326,2017-11-09,0,3,0,10659,...,49934,49657,0.001978,0.001747,0.000103,0.000058,0.001538,0.001755,1.510186e+09,1
3,303695,12,1,13,340,2017-11-09,0,3,0,1,...,3,2,0.000000,0.001747,0.000103,0.000085,0.000000,0.001490,1.510186e+09,1
4,173807,3,1,37,211,2017-11-09,0,3,0,10,...,148,40,0.000000,0.001747,0.000326,0.000500,0.002797,0.003148,1.510186e+09,1


In [39]:
%%time
full_df = train.append(valid).reset_index(drop=True)

CPU times: user 6.22 s, sys: 5.48 s, total: 11.7 s
Wall time: 11.7 s


In [42]:
full_df.shape

(53016937, 23)

In [43]:
del train,valid
gc.collect()

92

In [44]:
%%time
full_df["user_prev_date"] = full_df.groupby(['ip','device','os'])["ordinal_date"].shift(1)
full_df["user_date_diff"] = full_df["ordinal_date"] - full_df["user_prev_date"]

CPU times: user 7.3 s, sys: 1.84 s, total: 9.14 s
Wall time: 9.14 s


In [45]:
gc.collect()
full_df.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,...,mean_attr_by_iposdevice,device_attr_avg,app_attr_avg,channel_attr_avg,ip_attr_avg,os_attr_avg,ordinal_date,train_set,user_prev_date,user_date_diff
0,201143,11,1,13,487,2017-11-09,0,3,0,13,...,0.000000,0.001747,0.001551,0.001501,0.001064,0.001490,1.510186e+09,1,NaN,NaN
1,106107,14,1,19,439,2017-11-09,0,3,0,34,...,0.001289,0.001747,0.000247,0.000447,0.002257,0.001755,1.510186e+09,1,NaN,NaN
2,73516,12,1,19,326,2017-11-09,0,3,0,10659,...,0.001978,0.001747,0.000103,0.000058,0.001538,0.001755,1.510186e+09,1,NaN,NaN
3,303695,12,1,13,340,2017-11-09,0,3,0,1,...,0.000000,0.001747,0.000103,0.000085,0.000000,0.001490,1.510186e+09,1,NaN,NaN
4,173807,3,1,37,211,2017-11-09,0,3,0,10,...,0.000000,0.001747,0.000326,0.000500,0.002797,0.003148,1.510186e+09,1,NaN,NaN


In [53]:
%%time
gdf = full_df.groupby(['ip','device','os'])["ordinal_date"].agg(["min", "mean", "max", "std"]).reset_index()
gdf.columns = ["ip","device","os", "user_min_date", "user_mean_date", "user_max_date", "user_std_date"]
full_df = pd.merge(full_df, gdf, on=['ip','device','os'])
del gdf
gc.collect()


CPU times: user 50.2 s, sys: 12.9 s, total: 1min 3s
Wall time: 1min 2s


In [55]:
print(full_df.shape)
full_df.head()

(53016937, 29)


,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,...,ip_attr_avg,os_attr_avg,ordinal_date,train_set,user_prev_date,user_date_diff,user_min_date,user_mean_date,user_max_date,user_std_date
0,201143,11,1,13,487,2017-11-09 00:00:00,0,3,0,13,...,0.001064,0.00149,1.510186e+09,1,NaN,NaN,1.510186e+09,1.510212e+09,1.510238e+09,18371.435602
1,201143,15,1,13,111,2017-11-09 00:00:01,0,3,0,11,...,0.001064,0.00149,1.510186e+09,1,1.510186e+09,1.0,1.510186e+09,1.510212e+09,1.510238e+09,18371.435602
2,201143,3,1,13,379,2017-11-09 00:00:12,0,3,0,35,...,0.001064,0.00149,1.510186e+09,1,1.510186e+09,11.0,1.510186e+09,1.510212e+09,1.510238e+09,18371.435602
3,201143,18,1,13,107,2017-11-09 00:03:54,0,3,0,12,...,0.001064,0.00149,1.510186e+09,1,1.510186e+09,222.0,1.510186e+09,1.510212e+09,1.510238e+09,18371.435602
4,201143,3,1,13,280,2017-11-09 00:03:55,0,3,0,35,...,0.001064,0.00149,1.510186e+09,1,1.510186e+09,1.0,1.510186e+09,1.510212e+09,1.510238e+09,18371.435602


In [56]:
%%time
gdf = full_df[full_df['is_attributed']==1]
gdf = gdf.groupby(['ip','device','os'])["ordinal_date"].agg(["mean","std"]).reset_index()
gdf.columns = ["ip","device","os","user_mean_date_click", "user_std_date_click"]
full_df = pd.merge(full_df, gdf, on=['ip','device','os'], how="left")
del gdf
gc.collect()

CPU times: user 15.7 s, sys: 32.1 s, total: 47.8 s
Wall time: 47.8 s


In [59]:
full_df.head()

,ip,app,device,os,channel,click_time,is_attributed,dow,hour,clicks_by_ip_os_device_app,...,ordinal_date,train_set,user_prev_date,user_date_diff,user_min_date,user_mean_date,user_max_date,user_std_date,user_mean_date_click,user_std_date_click
0,201143,11,1,13,487,2017-11-09 00:00:00,0,3,0,13,...,1.510186e+09,1,NaN,NaN,1.510186e+09,1.510212e+09,1.510238e+09,18371.435602,NaN,NaN
1,201143,15,1,13,111,2017-11-09 00:00:01,0,3,0,11,...,1.510186e+09,1,1.510186e+09,1.0,1.510186e+09,1.510212e+09,1.510238e+09,18371.435602,NaN,NaN
2,201143,3,1,13,379,2017-11-09 00:00:12,0,3,0,35,...,1.510186e+09,1,1.510186e+09,11.0,1.510186e+09,1.510212e+09,1.510238e+09,18371.435602,NaN,NaN
3,201143,18,1,13,107,2017-11-09 00:03:54,0,3,0,12,...,1.510186e+09,1,1.510186e+09,222.0,1.510186e+09,1.510212e+09,1.510238e+09,18371.435602,NaN,NaN
4,201143,3,1,13,280,2017-11-09 00:03:55,0,3,0,35,...,1.510186e+09,1,1.510186e+09,1.0,1.510186e+09,1.510212e+09,1.510238e+09,18371.435602,NaN,NaN


In [62]:
full_df = full_df.drop(["ordinal_date", "click_time", "user_prev_date"], axis=1)

(53016937, 28)

In [64]:
gc.collect()
train = full_df[full_df["train_set"] == 1]
valid = full_df[full_df["train_set"] == 0]
train.shape,valid.shape

((49990379, 28), (3026558, 28))

In [65]:
del full_df
gc.collect()

0

In [ ]:
'''
def get_deltatime(df):
    GROUP_BY_NEXT_CLICKS = [
        {'groupby': ['ip','device','os']},
       ]

    # Calculate the time to next click for each group
    for spec in GROUP_BY_NEXT_CLICKS:

        # Name of new feature
        new_feature = '{}_nextClick'.format('_'.join(spec['groupby']))    

        # Unique list of features to select
        all_features = spec['groupby'] + ['click_time']

        # Run calculation
        print(f">> Grouping by {spec['groupby']}, and saving time to next click in: {new_feature}")
        df[new_feature] = df.groupby(spec['groupby']).click_time.transform(lambda x: x.diff().shift(-1)).dt.seconds
        
    df.drop(['click_time'],axis=1,inplace=True)
    return df

'''

In [68]:
train.drop(['ip','train_set'],axis=1,inplace=True)
valid.drop(['ip','train_set'],axis=1,inplace=True)
print(train.shape,valid.shape)
gc.collect()

(49990379, 26) (3026558, 26)


0

In [71]:
#test.reset_index(inplace=True,drop=True)
#valid.drop(['is_attributed'],axis=1,inplace=True)
#gc.collect()

74

In [ ]:
    
dtrain = lgbm.Dataset(train.drop(['is_attributed'],axis=1), label=train.is_attributed)
dvalid = lgbm.Dataset(x_valid, label=y_valid)

In [ ]:
#del x_train,x_valid,y_train,y_valid

In [73]:
gc.collect()

0

In [ ]:

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.1,
    'num_leaves':15,  
    'max_depth': 4,
    #'scale_pos_weight': 99.7
    }


evals_results = {}

print("Training the model...")

lgb_model = lgbm.train(params, dtrain,valid_sets=[dtrain, dvalid], 
                 valid_names=['train','valid'], 
                 evals_result=evals_results, 
                 num_boost_round=20)

In [ ]:
print('Feature names:', lgb_model.feature_name())
print('Feature importances:', list(lgb_model.feature_importance()))


In [ ]:
submit = pd.read_csv(path_test, dtype='int', usecols=['click_id'])
submit['is_attributed'] = lgb_model.predict(test[predictors], num_iteration=lgb_model.best_iteration)
submit.to_csv('submission.csv', index=False)


In [ ]:

del train
del val
gc.collect()
